In [5]:
# %% [markdown]
# Mapa HTML básico (sem hover de pixel) — raster com QML + favelas com tooltip

# %%
import json, math
from pathlib import Path

import numpy as np
import geopandas as gpd
import rasterio
import matplotlib.pyplot as plt
from rasterstats import zonal_stats
from xml.etree import ElementTree as ET

from rasterio.enums import Resampling
from rasterio.warp import calculate_default_transform, reproject
import folium

from branca.element import MacroElement, Element
from jinja2 import Template

from folium import plugins


# ---------- caminhos ----------
root_dir = Path.cwd().resolve().parent
data_dir = root_dir / "data"
docs_dir = root_dir / "docs"
docs_dir.mkdir(exist_ok=True)

caminho_geotiff = data_dir / "raster_html.tif"
caminho_estilo  = data_dir / "mapa_base.qml"
caminho_favelas = data_dir / "all_favelas_sp_2022.gpkg"

saida_png   = docs_dir / "raster_renderizado.png"  # ficará ao lado do HTML
saida_html  = docs_dir / "mapa.html"

# ---------- utils: ler QML e renderizar raster ----------
def ler_paleta_qml(qml_path):
    tree = ET.parse(qml_path)
    root = tree.getroot()
    renderer = root.find(".//rasterrenderer")
    if renderer is None or renderer.attrib.get("type") != "singlebandpseudocolor":
        raise ValueError("O QML precisa ser 'singlebandpseudocolor' (QGIS).")
    shader = renderer.find(".//colorrampshader")
    if shader is None:
        raise ValueError("QML sem <colorrampshader>.")

    shader_type = shader.attrib.get("colorRampType", "INTERPOLATED").upper()
    itens = []
    for it in shader.findall("./item"):
        val = it.attrib.get("value")
        col = it.attrib.get("color")
        lab = it.attrib.get("label", "")
        if val is None or col is None:
            continue
        if "," in col:
            r,g,b,*_ = [int(x) for x in col.split(",")]
            col_hex = "#{:02x}{:02x}{:02x}".format(r,g,b)
        else:
            col_hex = col
        try:
            val_f = float(val)
        except:
            val_f = float('nan')
        itens.append({"value": val_f, "color": col_hex, "label": lab})

    have_numeric = all(not math.isnan(i["value"]) for i in itens)
    if have_numeric:
        itens.sort(key=lambda d: d["value"])

    modo = "rampa" if shader_type == "INTERPOLATED" else "discreto"
    return itens, modo

def reprojectar_para_4326(caminho_tif):
    with rasterio.open(caminho_tif) as src:
        dst_crs = "EPSG:4326"
        transform, width, height = calculate_default_transform(
            src.crs, dst_crs, src.width, src.height, *src.bounds
        )
        data_dst = np.empty((height, width), dtype=src.dtypes[0])
        data_dst.fill(src.nodata if src.nodata is not None else 0)
        reproject(
            source=rasterio.band(src, 1),
            destination=data_dst,
            src_transform=src.transform,
            src_crs=src.crs,
            dst_transform=transform,
            dst_crs=dst_crs,
            resampling=Resampling.nearest
        )
        nodata = src.nodata
    return data_dst, transform, nodata

def aplicar_paleta(data, classes, modo, nodata=None):
    h, w = data.shape
    rgba = np.zeros((h, w, 4), dtype=np.uint8)
    if nodata is not None:
        mask_valid = (data != nodata) & ~np.isnan(data)
    else:
        mask_valid = ~np.isnan(data)

    def hex_to_rgb(hex_color):
        hex_color = hex_color.lstrip("#")
        return tuple(int(hex_color[i:i+2], 16) for i in (0, 2, 4))

    if modo == "discreto":
        values = [c["value"] for c in classes]
        have_numeric = all(not math.isnan(v) for v in values)
        if have_numeric:
            thresholds = values[:]
            colors = [hex_to_rgb(c["color"]) for c in classes]
            arr = data.astype(float).copy()
            arr[~mask_valid] = np.nan
            r = np.zeros_like(arr, dtype=np.uint8)
            g = np.zeros_like(arr, dtype=np.uint8)
            b = np.zeros_like(arr, dtype=np.uint8)
            a = np.zeros_like(arr, dtype=np.uint8)
            for i, thr in enumerate(thresholds):
                idx = (arr <= thr) & ~np.isnan(arr)
                r[idx], g[idx], b[idx], a[idx] = colors[i][0], colors[i][1], colors[i][2], 255
            rgba[...,0] = np.nan_to_num(r, nan=0).astype(np.uint8)
            rgba[...,1] = np.nan_to_num(g, nan=0).astype(np.uint8)
            rgba[...,2] = np.nan_to_num(b, nan=0).astype(np.uint8)
            rgba[...,3] = np.nan_to_num(a, nan=0).astype(np.uint8)
        else:
            r,g,b = hex_to_rgb(classes[0]["color"]) if classes else (200,200,200)
            rgba[...,0], rgba[...,1], rgba[...,2], rgba[...,3] = r,g,b,255
            rgba[~mask_valid,3] = 0
    else:
        vals, rgbs = [], []
        for c in classes:
            if not math.isnan(c["value"]):
                vals.append(float(c["value"]))
                rgbs.append(hex_to_rgb(c["color"]))
        if len(vals) < 2:
            r,g,b = rgbs[0] if rgbs else (200,200,200)
            rgba[...,0], rgba[...,1], rgba[...,2], rgba[...,3] = r,g,b,255
            rgba[~mask_valid,3] = 0
        else:
            vals = np.array(vals); rgbs = np.array(rgbs)
            arr = data.astype(float); arr[~mask_valid] = np.nan
            vmin, vmax = float(vals.min()), float(vals.max())
            norm = (arr - vmin) / (vmax - vmin); norm = np.clip(norm, 0, 1)
            r = np.interp(norm, (vals - vmin)/(vmax - vmin), rgbs[:,0])
            g = np.interp(norm, (vals - vmin)/(vmax - vmin), rgbs[:,1])
            b = np.interp(norm, (vals - vmin)/(vmax - vmin), rgbs[:,2])
            rgba[...,0] = np.nan_to_num(r, nan=0).astype(np.uint8)
            rgba[...,1] = np.nan_to_num(g, nan=0).astype(np.uint8)
            rgba[...,2] = np.nan_to_num(b, nan=0).astype(np.uint8)
            rgba[...,3] = (mask_valid * 255).astype(np.uint8)
    rgba[~mask_valid, 3] = 0
    return rgba

# ---------- ler dados ----------
gdf_favelas = gpd.read_file(caminho_favelas)

with rasterio.open(caminho_geotiff) as src:
    raster_crs = src.crs
    raster_nodata = src.nodata

# projeta favelas pro CRS do raster (pra calcular média sem distorção)
if gdf_favelas.crs != raster_crs:
    gdf_favelas = gdf_favelas.to_crs(raster_crs)

# ---------- média por polígono ----------
stats = zonal_stats(
    vectors=gdf_favelas.geometry,
    raster=str(caminho_geotiff),
    stats=["mean"],
    nodata=raster_nodata,
    all_touched=False
)
gdf_favelas["temp_media"] = [s["mean"] if s["mean"] is not None else float('nan') for s in stats]

# projeta para 4326 pro folium e prepara GeoJSON embutido
gdf_4326 = gdf_favelas.to_crs(4326)
gj = json.loads(gdf_4326.to_json())

# ---------- renderizar raster em 4326 com QML ----------
classes_qml, modo_qml = ler_paleta_qml(caminho_estilo)
raster_4326, transform_4326, _ = reprojectar_para_4326(caminho_geotiff)
rgba_img = aplicar_paleta(raster_4326, classes_qml, modo_qml, nodata=raster_nodata)

plt.imsave(saida_png, rgba_img)

# bounds para o overlay
height, width = raster_4326.shape
minx, miny = transform_4326 * (0, height)
maxx, maxy = transform_4326 * (width, 0)
bounds_latlon = [[miny, minx], [maxy, maxx]]
centro_lat = (miny + maxy) / 2
centro_lon = (minx + maxx) / 2

# ---------- montar o mapa ----------
m = folium.Map(
    location=[centro_lat, centro_lon],
    zoom_start=11,
    tiles=None,
    control_scale=True
)

# basemap sem rótulos
folium.TileLayer(
    tiles="https://{s}.basemaps.cartocdn.com/light_nolabels/{z}/{x}/{y}{r}.png",
    attr='&copy; CARTO | &copy; OpenStreetMap',
    name="Basemap sem rótulos",
    overlay=False,
    control=False
).add_to(m)

# raster renderizado (PNG) — caminho relativo ao HTML (ambos em docs/)
folium.raster_layers.ImageOverlay(
    name="Mapa LST",
    image=str(saida_png),
    bounds=bounds_latlon,
    opacity=0.75,
    interactive=False
).add_to(m)

# favelas — sem preenchimento, linha mais grossa + tooltip custom
def tooltip_html(props):
    nm = props.get("nm_fcu", "sem nome")
    tm = props.get("temp_media", None)
    is_nan = isinstance(tm, float) and math.isnan(tm)
    tm_fmt = "—" if tm is None or is_nan else f"{tm:.2f}"
    return f"""
    <div style="font-family:Arial; font-size:12px; line-height:1.25;">
      <b>Favela:</b><br/>{nm}<br/><br/>
      <b>Temperatura média (°C):</b><br/>{tm_fmt}
    </div>
    """

favelas_group = folium.FeatureGroup(name="Favelas").add_to(m)

def style_fn(feat):
    return {"color": "#222222", "weight": 2, "fill": False}

def highlight_fn(feat):
    return {"color": "#000000", "weight": 5.0}

# adiciona feição por feição para tooltip HTML custom
for feat in gj["features"]:
    props = feat.get("properties", {})
    folium.GeoJson(
        data=feat,
        style_function=style_fn,
        highlight_function=highlight_fn,
        tooltip=folium.Tooltip(tooltip_html(props), sticky=True, direction="top")
    ).add_to(favelas_group)

folium.LayerControl(position="topleft", collapsed=False).add_to(m)


# === LEGENDA AUTOMÁTICA (a partir do QML) ===

def construir_legenda_html(classes, modo, titulo="Temperatura LST", unidade="°C"):
    # tenta detectar se todos os valores são numéricos
    valores = [c.get("value") for c in classes]
    have_numeric = len(valores) > 0 and all(isinstance(v, (int,float)) and not math.isnan(v) for v in valores)

    # rampa contínua (INTERPOLATED) → gera barra com gradient
    if modo == "rampa" and have_numeric and len(classes) >= 2:
        # garante ordenação por valor
        classes_ord = sorted(classes, key=lambda d: float(d["value"]))
        vmin, vmax = float(classes_ord[0]["value"]), float(classes_ord[-1]["value"])
        # stops de cor (0%..100%)
        n = len(classes_ord)
        stops = []
        for i, c in enumerate(classes_ord):
            pct = (i/(n-1))*100 if n > 1 else 0
            stops.append(f"{c['color']} {pct:.2f}%")
        gradient_css = ", ".join(stops)

        html = f"""
        <div id="map-legend" style="
          position:absolute; bottom:60px; left:12px; z-index:9999;
          background:rgba(255,255,255,0.92); padding:10px 12px; border-radius:8px;
          box-shadow:0 0 8px rgba(0,0,0,0.2); font:12px/1.25 Arial, sans-serif;">
          <div style="font-weight:600; margin-bottom:8px;">{titulo} ({unidade})</div>
          <div style="display:flex; align-items:center; gap:10px;">
            <div style="
              width:16px; height:100px; border:1px solid #00000033;
              background: linear-gradient(to top, {gradient_css});
            "></div>
            <div style="display:flex; flex-direction:column; justify-content:space-between; height:100px;">
              <div>{vmax:.2f}</div>
              <div style="text-align:right;">{vmin:.2f}</div>
            </div>
          </div>
        </div>
        """
        return html

    # classes discretas → lista de amostras (cor + rótulo/valor)
    linhas = []
    for c in classes:
        cor = c.get("color", "#888")
        rot = c.get("label") or (f"{float(c['value']):.2f}" if c.get("value") is not None and not math.isnan(c["value"]) else "")
        linhas.append(f"""
          <div class="leg-row" style="display:flex; align-items:center; gap:6px; margin:2px 0;">
            <span style="width:14px; height:14px; border:1px solid #00000022; background:{cor}; display:inline-block;"></span>
            <span>{rot}</span>
          </div>
        """)
    html = f"""
    <div id="map-legend" style="
      position:absolute; bottom:12px; left:12px; z-index:9999;
      max-height:220px; overflow:auto;
      background:rgba(255,255,255,0.92); padding:10px 12px; border-radius:8px;
      box-shadow:0 0 8px rgba(0,0,0,0.2); font:12px/1.25 Arial, sans-serif;">
      <div style="font-weight:600; margin-bottom:6px;">{titulo} ({unidade})</div>
      {''.join(linhas)}
    </div>
    """
    return html

# monta e insere no mapa (posição: canto inferior esquerdo, pra não brigar com a caixinha do hover)
html_legenda = construir_legenda_html(classes_qml, modo_qml, titulo="Temperatura LST", unidade="°C")
m.get_root().html.add_child(Element(html_legenda))


# --- HOVER DE PIXEL (valor do raster) via API local ---

api_url_point = "https://cefavela-mapa-interativo-calor.onrender.com/point"
api_url_zonal = "https://cefavela-mapa-interativo-calor.onrender.com/zonal"

tpl_point = Template("""
{% macro script(this, kwargs) %}
(function() {
  var apiUrl = {{ this.api_url_point | tojson }};
  var mapObj = {{ this._parent.get_name() }};

  // caixinha no canto
  var box = L.control({position:'topleft'});
  box.onAdd = function(){
    var div = L.DomUtil.create('div','leaflet-control');
    div.style.background='rgba(255,255,255,0.9)';
    div.style.padding='6px 8px';
    div.style.borderRadius='6px';
    div.style.boxShadow='0 0 6px rgba(0,0,0,0.2)';
    div.style.font='12px/1.2 Arial, sans-serif';
    div.innerHTML='<b>Temperatura:</b> —';
    this._div = div; return div;
  };
  box.addTo(mapObj);

  function setVal(v){
    box._div.innerHTML = '<b>Temperatura (°C):</b> ' + (v==null?'—':Number(v).toFixed(2));
  }

  // debounce para reduzir chamadas
  let timer = null;
  function debouncedFetch(lat,lng){
    if (timer) clearTimeout(timer);
    timer = setTimeout(async () => {
      try{
        const resp = await fetch(apiUrl, {
          method:'POST',
          headers:{'Content-Type':'application/json'},
          body: JSON.stringify({lat:lat, lon:lng})
        });
        const js = await resp.json();
        setVal(js.value);
      }catch(e){
        setVal(null);
      }
    }, 80);
  }

  mapObj.on('mousemove', function(e){
    debouncedFetch(e.latlng.lat, e.latlng.lng);
  });
})();
{% endmacro %}
""")

class HoverProbe(MacroElement):
    def __init__(self, api_url_point):
        super().__init__()
        self._template = tpl_point
        self.api_url_point = api_url_point

hp = HoverProbe(api_url_point)
m.add_child(hp)

plugins.Draw(
    export=False,
    position="topleft",
    draw_options={
        "polyline": False,
        "circle": False,
        "circlemarker": False,
        "marker": False,
        "polygon": True,
        "rectangle": True
    },
    edit_options={"edit": False, "remove": True}
).add_to(m)

tpl_zonal = Template("""
{% macro script(this, kwargs) %}
(function() {
  var mapObj = {{ this._parent.get_name() }};
  var zonalUrl = {{ this.api_url_zonal | tojson }};

  function showPopupAt(layer, text){
    try{
      var center = layer.getBounds ? layer.getBounds().getCenter() : layer.getLatLng();
      L.popup().setLatLng(center).setContent(text).openOn(mapObj);
    }catch(e){ alert(text); }
  }

  mapObj.on(L.Draw.Event.CREATED, async function (e) {
    var layer = e.layer;
    var gj = layer.toGeoJSON(); // já vem em coordenadas [lon,lat] (GeoJSON)
    try{
      const resp = await fetch(zonalUrl, {
        method:'POST',
        headers:{'Content-Type':'application/json'},
        body: JSON.stringify({ geometry: gj.geometry })
      });
      const js = await resp.json();
      var msg = (js.mean==null)
        ? "<b>Temperatura média (°C): </b> —"
        : "<b>Temperatura média (°C): </b> " + Number(js.mean).toFixed(2);
      showPopupAt(layer, msg);
    }catch(err){
      showPopupAt(layer, "<b>Erro:</b> não foi possível calcular.");
    }
    layer.addTo(mapObj); // mantém o desenho no mapa
  });
})();
{% endmacro %}
""")

class ZonalProbe(MacroElement):
    def __init__(self, api_url_zonal):
        super().__init__()
        self._template = tpl_zonal
        self.api_url_zonal = api_url_zonal

m.add_child(ZonalProbe(api_url_zonal))

# salva no docs/
m.save(str(saida_html))
print(f"OK! Abra: {saida_html}")


OK! Abra: C:\Users\bianc\OneDrive\CEFAVELA UFABC\Artigo Nexo\cefavela_mapa_interativo_calor\docs\mapa.html
